Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE:
MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

MindSPONGE advanced tutorial 04: Hybrid enhanced sampling and MetaITS

In [1]:
from mindspore import context
from sponge import Sponge
from sponge import ForceField
from sponge import set_global_units
from sponge import Protein
from sponge import WithEnergyCell
from sponge import UpdaterMD
from sponge.optimizer import SteepestDescent
from sponge.control import VelocityVerlet
from sponge.callback import WriteH5MD, RunInfo
from sponge.control import Langevin
from sponge.sampling import Metadynamics, ITS
from sponge.function import VelocityGenerator
from sponge.colvar import Torsion
from sponge.function import PI

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")
set_global_units('nm', 'kj/mol')

In [3]:
system = Protein(pdb='alad.pdb')
potential = ForceField(system, 'AMBER.FF14SB')

In [4]:
phi = Torsion([4, 6, 8, 14])
psi = Torsion([6, 8, 14, 16])

In [5]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)
mini = Sponge(system, potential, min_opt, metrics={'phi': phi, 'psi': psi})

[MindSPONGE] Using fully connected neighbour list (not updated).


In [6]:
run_info = RunInfo(10)
mini.run(100, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-05-18 15:41:46
[MindSPONGE] Step: 0, E_pot: -55.75643, phi: -3.140886, psi: -3.1410766
[MindSPONGE] Step: 10, E_pot: -58.40555, phi: -3.1340268, psi: -3.1368413
[MindSPONGE] Step: 20, E_pot: -60.168476, phi: -3.1277595, psi: -3.1337605
[MindSPONGE] Step: 30, E_pot: -61.549706, phi: -3.1221528, psi: -3.131402
[MindSPONGE] Step: 40, E_pot: -62.696198, phi: -3.1171532, psi: -3.1295478
[MindSPONGE] Step: 50, E_pot: -63.674084, phi: -3.112676, psi: -3.1280725
[MindSPONGE] Step: 60, E_pot: -64.52254, phi: -3.1086385, psi: -3.1268952
[MindSPONGE] Step: 70, E_pot: -65.26819, phi: -3.1049676, psi: -3.1259575
[MindSPONGE] Step: 80, E_pot: -65.93031, phi: -3.1016028, psi: -3.1252148
[MindSPONGE] Step: 90, E_pot: -66.52346, phi: -3.0984938, psi: -3.1246324
[MindSPONGE] Finished simulation at 2023-05-18 15:41:49
[MindSPONGE] Simulation time: 3.73 seconds.
--------------------------------------------------------------------------------


In [7]:
metad = Metadynamics(
    colvar=[phi, psi],
    update_pace=10,
    height=2.5,
    sigma=0.05,
    grid_min=-PI,
    grid_max=PI,
    grid_bin=360,
    temperature=300,
    bias_factor=100,
)

In [8]:
its = ITS(
    sim_temp=300,
    temp_min=270,
    temp_max=670,
    temp_bin=200,
    update_pace=100,
    unlinear_temp=True,
)

In [9]:
sim = WithEnergyCell(system, potential, bias=metad, wrapper=its)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [10]:
vgen = VelocityGenerator(300)
velocity = vgen(system.shape, system.atom_mass)
opt = UpdaterMD(
    system,
    integrator=VelocityVerlet(system),
    thermostat=Langevin(system, 300),
    time_step=1e-3,
    velocity=velocity
)

In [11]:
md = Sponge(sim, optimizer=opt, metrics={'phi': phi, 'psi': psi})

In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_a04.h5md', save_freq=10)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-05-18 15:41:57
[MindSPONGE] Step: 0, E_pot: -79.74093, E_kin: 60.13574, E_tot: -19.60519, Temperature: 229.60847, phi: -3.0927696, psi: -3.1155593
[MindSPONGE] Step: 10, E_pot: -119.28449, E_kin: 44.476517, E_tot: -74.807976, Temperature: 169.8189, phi: -3.037634, psi: -3.017646
[MindSPONGE] Step: 20, E_pot: -118.148506, E_kin: 44.38232, E_tot: -73.76619, Temperature: 169.45926, phi: -2.9700854, psi: -2.9923446
[MindSPONGE] Step: 30, E_pot: -114.2139, E_kin: 44.84871, E_tot: -69.36519, Temperature: 171.24, phi: -2.921324, psi: -3.0061398
[MindSPONGE] Step: 40, E_pot: -126.56792, E_kin: 55.20807, E_tot: -71.35985, Temperature: 210.79381, phi: -2.8557289, psi: -3.11261
[MindSPONGE] Step: 50, E_pot: -125.0885, E_kin: 59.35125, E_tot: -65.73725, Temperature: 226.61316, phi: -2.839629, psi: 3.0605183
[MindSPONGE] Step: 60, E_pot: -119.36946, E_kin: 51.74867, E_tot: -67.62079, Temperature: 197.58522, phi: -2.8968134, psi: 2.9166963
[MindSPONGE] Step: 7